In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pickle

In [2]:
# path to playlist file
root_path = !pwd
root_path = str(root_path[0])

In [3]:
# filename 
filepath = f"{root_path}/playlist_data/sampledata/"
filename = 'mpd.slice.0-999.json'

#filepath = f"{root_path}/playlist_data/"
#filename = 'challenge_set.json'

In [4]:
# path + filename
fpath_name = f"{filepath}{filename}"
# print(fpath_name)

In [5]:
# Open file of playlist and obtain fields of json file 
with open(fpath_name) as data_file:    
    data = json.load(data_file)  

In [6]:
# # Explore fields in json file
# print("File keys: ", list(data.keys()))
# print("info: ",data[list(data.keys())[0]])
# #print(f"{list(data.keys())[1]} : ",data[list(data.keys())[1]])
# print("playlists: ",list(data['playlists'][0].keys()))
# print("tracks: ", list(data['playlists'][0]['tracks'][0].keys()))

In [7]:
# Extract track features
num_keys_old = 0
for index, playlist in enumerate(data['playlists']):
    num_keys =  len(playlist.keys())
    if index == 0:
        num_key_old = num_keys
        continue
    if num_keys > num_key_old:
        keys = playlist.keys()
    num_key_old = num_keys

In [8]:
# Construct keys of playlist
keys = list(keys)
keys.remove('tracks')
print(" Playlist keys: ", keys)

 Playlist keys:  ['name', 'collaborative', 'pid', 'modified_at', 'num_tracks', 'num_albums', 'num_followers', 'num_edits', 'duration_ms', 'num_artists', 'description']


In [9]:
# Create dataframe with track and playlist info
music_df = pd.json_normalize(data['playlists'],  meta = keys, meta_prefix = 'playlist_', errors='ignore', record_path=['tracks'], record_prefix = 'track_')
music_df.head()

,track_pos,track_artist_name,track_track_uri,track_artist_uri,track_track_name,track_album_uri,track_duration_ms,track_album_name,playlist_name,playlist_collaborative,playlist_pid,playlist_modified_at,playlist_num_tracks,playlist_num_albums,playlist_num_followers,playlist_num_edits,playlist_duration_ms,playlist_num_artists,playlist_description
0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
2,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
3,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
4,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN


In [10]:
music_df.shape

(67503, 19)

In [11]:
# Check column values 
music_df[music_df['playlist_pid'] == 0].columns

Index(['track_pos', 'track_artist_name', 'track_track_uri', 'track_artist_uri',
       'track_track_name', 'track_album_uri', 'track_duration_ms',
       'track_album_name', 'playlist_name', 'playlist_collaborative',
       'playlist_pid', 'playlist_modified_at', 'playlist_num_tracks',
       'playlist_num_albums', 'playlist_num_followers', 'playlist_num_edits',
       'playlist_duration_ms', 'playlist_num_artists', 'playlist_description'],
      dtype='object')

In [12]:
subset_df = music_df

In [13]:
len(subset_df)

67503

In [14]:
# Select indexes of playlists that have a description
# playlists_id_with_description = music_df[~music_df['playlist_description'].isna()]['playlist_pid'].unique()
# print(playlists_id_with_description)

In [15]:
# Working only with playlists that contain a "description"
# subset_df = music_df[music_df['playlist_pid'].isin(playlists_id_with_description)]
# print(subset_df['playlist_pid'].unique())

In [16]:
num_records = len(subset_df)
num_unique_records = len(subset_df[['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x),axis=1).unique())
print('Number of table records: ', num_records)
print('Number of unique artist, song pairs: ', num_unique_records )
print('Number of repeated songs: ', num_records - num_unique_records )
print('Number of playlists: ', len(subset_df['playlist_pid'].unique()))
print('Number of unique artists: ', len(subset_df['track_artist_name'].unique()))

Number of table records:  67503
Number of unique artist, song pairs:  34250
Number of repeated songs:  33253
Number of playlists:  1000
Number of unique artists:  9722


# I have to put my IDs manually in here

In [17]:

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id = ''
client_secret = ''

spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [18]:
# Function to select track_uri ONLY Nrecords records for now
Nrecords = 500
def get_features(df, Nrecords):
    indexes = []
    audio_features = []
    track_popularity = []
    artist_genre = []
    artist_popularity = []
    album_popularity = []
    start = 0
    for row_index, row in df.iloc[0:Nrecords].iterrows():
        #print(row_index, row['track_track_uri'])
        track_uri = row['track_track_uri']
        artist_uri = row['track_artist_uri']
        album_uri = row['track_album_uri']
        if start == 0:
            keys = spotify.audio_features(tracks=track_uri)[0].keys()
        start += 1
    
        track_popularity.append(spotify.track(track_uri)['popularity'])
        
        indexes.append(row_index) 
        audio_features.append(spotify.audio_features(tracks=track_uri)[0].values())
        
        artist_results = spotify.artist(artist_uri)
        
        artist_genre.append(artist_results['genres'])
        artist_popularity.append(artist_results['popularity'])
        #print(artist_results['genres'], artist_results['popularity'])
        album_results = spotify.album(album_uri)
        album_popularity.append(album_results['popularity'])
        
        
    features_df = pd.DataFrame(audio_features, columns=keys)
    features_df['song_popularity'] = track_popularity
    features_df['artist_genre'] = artist_genre
    features_df['artist_popularity']  = artist_popularity
    features_df['album_popularity'] = album_popularity
    features_df['index'] = indexes
    features_df.set_index('index', inplace=True)
    return features_df

spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [19]:
audio_sp_df = get_features(subset_df , Nrecords)
audio_sp_df.index.name = None
audio_sp_df.drop(['type', 'id', 'track_href', 'analysis_url'], inplace=True, axis=1)
audio_sp_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity
0,0.904,0.813,4,-7.105,0,0.1210,0.03110,0.006970,0.0471,0.810,125.461,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,226864,4,69,"[dance pop, hip hop, hip pop, neo soul, pop ra...",72,62
1,0.774,0.838,5,-3.914,0,0.1140,0.02490,0.025000,0.2420,0.924,143.040,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,198800,4,84,"[dance pop, pop]",80,78
2,0.664,0.758,2,-6.583,0,0.2100,0.00238,0.000000,0.0598,0.701,99.259,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,235933,4,21,"[pop, r&b]",87,17
3,0.892,0.714,4,-6.055,0,0.1410,0.20100,0.000234,0.0521,0.817,100.972,spotify:track:1AWQoqb9bSvzTjaLralEkT,267267,4,79,"[dance pop, pop]",79,77
4,0.853,0.606,0,-4.596,1,0.0713,0.05610,0.000000,0.3130,0.654,94.759,spotify:track:1lzr43nnXAijIGYnCT8M8H,227600,4,4,"[dance pop, pop rap, reggae fusion]",73,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.543,0.793,0,-5.905,1,0.0403,0.22500,0.000114,0.3630,0.553,140.060,spotify:track:4YZFC2igC3mzuktFHSF9mY,208453,4,38,[modern southern rock],55,47
496,0.532,0.883,0,-2.743,0,0.0352,0.09510,0.000000,0.3040,0.700,99.995,spotify:track:17Vry8h1fUTYg8NL6pHhem,190240,4,0,"[contemporary country, country, country road, ...",59,0
497,0.595,0.653,7,-4.935,1,0.0265,0.24600,0.000000,0.2640,0.542,128.930,spotify:track:3oW6SWwGqiZSPTiAp7ZQoH,241533,4,75,"[contemporary country, country, country road, ...",64,69
498,0.579,0.801,0,-7.454,1,0.0449,0.12100,0.000076,0.0821,0.972,174.755,spotify:track:2LbOTztU3giUjjYmgtKgj3,237267,4,56,[country],72,61


In [20]:
# Parameters to use for the model
audio_sp_df.keys()

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'uri', 'duration_ms', 'time_signature', 'song_popularity',
       'artist_genre', 'artist_popularity', 'album_popularity'],
      dtype='object')

In [21]:
check_df = subset_df.iloc[0:Nrecords].merge(audio_sp_df, how='left' , left_on = 'track_track_uri', right_on='uri')
check_df.drop(['track_pos', 'uri', 'mode', 'playlist_duration_ms','playlist_num_albums','playlist_num_artists',  'track_artist_uri', 'track_album_uri', 'track_duration_ms','playlist_num_followers', 'playlist_num_edits', 'playlist_collaborative', 'playlist_modified_at', 'playlist_num_tracks'], inplace = True, axis=1)
check_df

,track_artist_name,track_track_uri,track_track_name,track_album_name,playlist_name,playlist_pid,playlist_description,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),The Cookbook,Throwbacks,0,NaN,0.904,0.813,4,...,0.006970,0.0471,0.810,125.461,226864,4,69,"[dance pop, hip hop, hip pop, neo soul, pop ra...",72,62
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Toxic,In The Zone,Throwbacks,0,NaN,0.774,0.838,5,...,0.025000,0.2420,0.924,143.040,198800,4,84,"[dance pop, pop]",80,78
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,0,NaN,0.664,0.758,2,...,0.000000,0.0598,0.701,99.259,235933,4,21,"[pop, r&b]",87,17
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,Rock Your Body,Justified,Throwbacks,0,NaN,0.892,0.714,4,...,0.000234,0.0521,0.817,100.972,267267,4,79,"[dance pop, pop]",79,77
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,Hot Shot,Throwbacks,0,NaN,0.853,0.606,0,...,0.000000,0.3130,0.654,94.759,227600,4,4,"[dance pop, pop rap, reggae fusion]",73,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,The Cadillac Three,spotify:track:4YZFC2igC3mzuktFHSF9mY,Drunk Like You,Bury Me In My Boots,old country,9,NaN,0.543,0.793,0,...,0.000114,0.3630,0.553,140.060,208453,4,38,[modern southern rock],55,47
518,Chris Lane,spotify:track:17Vry8h1fUTYg8NL6pHhem,Fix,Girl Problems,old country,9,NaN,0.532,0.883,0,...,0.000000,0.3040,0.700,99.995,190240,4,0,"[contemporary country, country, country road, ...",59,0
519,Travis Tritt,spotify:track:3oW6SWwGqiZSPTiAp7ZQoH,It's A Great Day To Be Alive,Down The Road I Go,old country,9,NaN,0.595,0.653,7,...,0.000000,0.2640,0.542,128.930,241533,4,75,"[contemporary country, country, country road, ...",64,69
520,Alan Jackson,spotify:track:2LbOTztU3giUjjYmgtKgj3,Chattahoochee - Extended Mix,16 Biggest Hits,old country,9,NaN,0.579,0.801,0,...,0.000076,0.0821,0.972,174.755,237267,4,56,[country],72,61


In [22]:
check_df.describe()

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_popularity,album_popularity
count,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000
mean,0.596734,0.680392,4.854406,-7.070588,0.075071,0.199901,0.083551,0.183640,0.519212,122.871651,234150.574713,3.955939,30.701149,61.049808,31.480843
std,0.163848,0.200619,3.619879,3.434959,0.074261,0.257980,0.222522,0.140149,0.224485,28.053671,64341.248057,0.277037,31.548317,19.272901,31.769513
min,0.084900,0.013400,0.000000,-29.917000,0.023200,0.000010,0.000000,0.026800,0.038500,48.973000,65306.000000,1.000000,0.000000,0.000000,0.000000
25%,0.490000,0.553250,1.000000,-8.801250,0.034425,0.011375,0.000000,0.089925,0.357000,100.977500,199752.250000,4.000000,0.000000,50.000000,0.000000
50%,0.615500,0.716000,5.000000,-6.319000,0.047150,0.073200,0.000043,0.129000,0.520000,120.496000,225113.500000,4.000000,23.000000,65.000000,27.000000
75%,0.717000,0.834000,8.000000,-4.670750,0.077200,0.310500,0.006282,0.237750,0.697500,140.307500,259736.750000,4.000000,61.750000,76.000000,64.000000
max,0.979000,0.986000,11.000000,-1.029000,0.505000,0.989000,0.991000,0.914000,0.972000,210.857000,658987.000000,5.000000,85.000000,91.000000,86.000000


In [23]:
# Make album popularity equal to artist popularity if album popularity is 0
check_df['album_popularity'] = np.where(check_df['album_popularity'] == 0, check_df['artist_popularity'], check_df['album_popularity'])
check_df['song_popularity'] = np.where(check_df['song_popularity'] == 0, check_df['artist_popularity'], check_df['song_popularity'])
check_df

,track_artist_name,track_track_uri,track_track_name,track_album_name,playlist_name,playlist_pid,playlist_description,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),The Cookbook,Throwbacks,0,NaN,0.904,0.813,4,...,0.006970,0.0471,0.810,125.461,226864,4,69,"[dance pop, hip hop, hip pop, neo soul, pop ra...",72,62
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Toxic,In The Zone,Throwbacks,0,NaN,0.774,0.838,5,...,0.025000,0.2420,0.924,143.040,198800,4,84,"[dance pop, pop]",80,78
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,0,NaN,0.664,0.758,2,...,0.000000,0.0598,0.701,99.259,235933,4,21,"[pop, r&b]",87,17
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,Rock Your Body,Justified,Throwbacks,0,NaN,0.892,0.714,4,...,0.000234,0.0521,0.817,100.972,267267,4,79,"[dance pop, pop]",79,77
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,Hot Shot,Throwbacks,0,NaN,0.853,0.606,0,...,0.000000,0.3130,0.654,94.759,227600,4,4,"[dance pop, pop rap, reggae fusion]",73,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,The Cadillac Three,spotify:track:4YZFC2igC3mzuktFHSF9mY,Drunk Like You,Bury Me In My Boots,old country,9,NaN,0.543,0.793,0,...,0.000114,0.3630,0.553,140.060,208453,4,38,[modern southern rock],55,47
518,Chris Lane,spotify:track:17Vry8h1fUTYg8NL6pHhem,Fix,Girl Problems,old country,9,NaN,0.532,0.883,0,...,0.000000,0.3040,0.700,99.995,190240,4,59,"[contemporary country, country, country road, ...",59,59
519,Travis Tritt,spotify:track:3oW6SWwGqiZSPTiAp7ZQoH,It's A Great Day To Be Alive,Down The Road I Go,old country,9,NaN,0.595,0.653,7,...,0.000000,0.2640,0.542,128.930,241533,4,75,"[contemporary country, country, country road, ...",64,69
520,Alan Jackson,spotify:track:2LbOTztU3giUjjYmgtKgj3,Chattahoochee - Extended Mix,16 Biggest Hits,old country,9,NaN,0.579,0.801,0,...,0.000076,0.0821,0.972,174.755,237267,4,56,[country],72,61


In [24]:
check_df.describe()

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_popularity,album_popularity
count,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000
mean,0.596734,0.680392,4.854406,-7.070588,0.075071,0.199901,0.083551,0.183640,0.519212,122.871651,234150.574713,3.955939,55.409962,61.049808,55.323755
std,0.163848,0.200619,3.619879,3.434959,0.074261,0.257980,0.222522,0.140149,0.224485,28.053671,64341.248057,0.277037,22.665991,19.272901,22.780699
min,0.084900,0.013400,0.000000,-29.917000,0.023200,0.000010,0.000000,0.026800,0.038500,48.973000,65306.000000,1.000000,0.000000,0.000000,0.000000
25%,0.490000,0.553250,1.000000,-8.801250,0.034425,0.011375,0.000000,0.089925,0.357000,100.977500,199752.250000,4.000000,42.250000,50.000000,43.000000
50%,0.615500,0.716000,5.000000,-6.319000,0.047150,0.073200,0.000043,0.129000,0.520000,120.496000,225113.500000,4.000000,60.000000,65.000000,61.000000
75%,0.717000,0.834000,8.000000,-4.670750,0.077200,0.310500,0.006282,0.237750,0.697500,140.307500,259736.750000,4.000000,73.000000,76.000000,73.000000
max,0.979000,0.986000,11.000000,-1.029000,0.505000,0.989000,0.991000,0.914000,0.972000,210.857000,658987.000000,5.000000,90.000000,91.000000,90.000000


In [25]:
check_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 522 entries, 0 to 521
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   track_artist_name     522 non-null    object 
 1   track_track_uri       522 non-null    object 
 2   track_track_name      522 non-null    object 
 3   track_album_name      522 non-null    object 
 4   playlist_name         522 non-null    object 
 5   playlist_pid          522 non-null    object 
 6   playlist_description  0 non-null      object 
 7   danceability          522 non-null    float64
 8   energy                522 non-null    float64
 9   key                   522 non-null    int64  
 10  loudness              522 non-null    float64
 11  speechiness           522 non-null    float64
 12  acousticness          522 non-null    float64
 13  instrumentalness      522 non-null    float64
 14  liveness              522 non-null    float64
 15  valence               5

In [26]:
# NLP workflow
# Need to vectorize string fields
# Use all string fields and combine them in a column containing all the words f
# from collections import Counter

In [27]:
check_df['artist_genre']

0      [dance pop, hip hop, hip pop, neo soul, pop ra...
1                                       [dance pop, pop]
2                                             [pop, r&b]
3                                       [dance pop, pop]
4                    [dance pop, pop rap, reggae fusion]
                             ...                        
517                               [modern southern rock]
518    [contemporary country, country, country road, ...
519    [contemporary country, country, country road, ...
520                                            [country]
521                            [outlaw country, redneck]
Name: artist_genre, Length: 522, dtype: object

In [28]:
#check_df['artist_genre'] = check_df['artist_genre'].apply(lambda x: ' '.join([i.replace('-', '_') for i in x]))
check_df['artist_genre'] = check_df['artist_genre'].apply(lambda x: ' '.join([i.replace('-', '_').replace('_', '') for i in x]))

In [29]:
check_df

,track_artist_name,track_track_uri,track_track_name,track_album_name,playlist_name,playlist_pid,playlist_description,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),The Cookbook,Throwbacks,0,NaN,0.904,0.813,4,...,0.006970,0.0471,0.810,125.461,226864,4,69,dance pop hip hop hip pop neo soul pop rap r&b...,72,62
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Toxic,In The Zone,Throwbacks,0,NaN,0.774,0.838,5,...,0.025000,0.2420,0.924,143.040,198800,4,84,dance pop pop,80,78
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,0,NaN,0.664,0.758,2,...,0.000000,0.0598,0.701,99.259,235933,4,21,pop r&b,87,17
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,Rock Your Body,Justified,Throwbacks,0,NaN,0.892,0.714,4,...,0.000234,0.0521,0.817,100.972,267267,4,79,dance pop pop,79,77
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,Hot Shot,Throwbacks,0,NaN,0.853,0.606,0,...,0.000000,0.3130,0.654,94.759,227600,4,4,dance pop pop rap reggae fusion,73,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,The Cadillac Three,spotify:track:4YZFC2igC3mzuktFHSF9mY,Drunk Like You,Bury Me In My Boots,old country,9,NaN,0.543,0.793,0,...,0.000114,0.3630,0.553,140.060,208453,4,38,modern southern rock,55,47
518,Chris Lane,spotify:track:17Vry8h1fUTYg8NL6pHhem,Fix,Girl Problems,old country,9,NaN,0.532,0.883,0,...,0.000000,0.3040,0.700,99.995,190240,4,59,contemporary country country country road mode...,59,59
519,Travis Tritt,spotify:track:3oW6SWwGqiZSPTiAp7ZQoH,It's A Great Day To Be Alive,Down The Road I Go,old country,9,NaN,0.595,0.653,7,...,0.000000,0.2640,0.542,128.930,241533,4,75,contemporary country country country road outl...,64,69
520,Alan Jackson,spotify:track:2LbOTztU3giUjjYmgtKgj3,Chattahoochee - Extended Mix,16 Biggest Hits,old country,9,NaN,0.579,0.801,0,...,0.000076,0.0821,0.972,174.755,237267,4,56,country,72,61


In [30]:
check_df['artist_genre'] = check_df['artist_genre'].apply(lambda x: ' '.join(list(set(x.split()))))

In [31]:
check_df

,track_artist_name,track_track_uri,track_track_name,track_album_name,playlist_name,playlist_pid,playlist_description,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),The Cookbook,Throwbacks,0,NaN,0.904,0.813,4,...,0.006970,0.0471,0.810,125.461,226864,4,69,dance urban hop hip virginia soul r&b pop cont...,72,62
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Toxic,In The Zone,Throwbacks,0,NaN,0.774,0.838,5,...,0.025000,0.2420,0.924,143.040,198800,4,84,dance pop,80,78
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,0,NaN,0.664,0.758,2,...,0.000000,0.0598,0.701,99.259,235933,4,21,r&b pop,87,17
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,Rock Your Body,Justified,Throwbacks,0,NaN,0.892,0.714,4,...,0.000234,0.0521,0.817,100.972,267267,4,79,dance pop,79,77
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,Hot Shot,Throwbacks,0,NaN,0.853,0.606,0,...,0.000000,0.3130,0.654,94.759,227600,4,4,dance reggae pop fusion rap,73,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,The Cadillac Three,spotify:track:4YZFC2igC3mzuktFHSF9mY,Drunk Like You,Bury Me In My Boots,old country,9,NaN,0.543,0.793,0,...,0.000114,0.3630,0.553,140.060,208453,4,38,modern southern rock,55,47
518,Chris Lane,spotify:track:17Vry8h1fUTYg8NL6pHhem,Fix,Girl Problems,old country,9,NaN,0.532,0.883,0,...,0.000000,0.3040,0.700,99.995,190240,4,59,country road contemporary rock modern,59,59
519,Travis Tritt,spotify:track:3oW6SWwGqiZSPTiAp7ZQoH,It's A Great Day To Be Alive,Down The Road I Go,old country,9,NaN,0.595,0.653,7,...,0.000000,0.2640,0.542,128.930,241533,4,75,country road contemporary outlaw,64,69
520,Alan Jackson,spotify:track:2LbOTztU3giUjjYmgtKgj3,Chattahoochee - Extended Mix,16 Biggest Hits,old country,9,NaN,0.579,0.801,0,...,0.000076,0.0821,0.972,174.755,237267,4,56,country,72,61


In [32]:
# recheck = check.apply(lambda x: list(set(x.split())))
# print(check)
# print(recheck)
# # check_df['artist_genre'] = check_df['artist_genre'].apply(lambda x: ''.join([i.replace('', '_') for i in x]))

In [33]:
check_df

,track_artist_name,track_track_uri,track_track_name,track_album_name,playlist_name,playlist_pid,playlist_description,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),The Cookbook,Throwbacks,0,NaN,0.904,0.813,4,...,0.006970,0.0471,0.810,125.461,226864,4,69,dance urban hop hip virginia soul r&b pop cont...,72,62
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Toxic,In The Zone,Throwbacks,0,NaN,0.774,0.838,5,...,0.025000,0.2420,0.924,143.040,198800,4,84,dance pop,80,78
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,0,NaN,0.664,0.758,2,...,0.000000,0.0598,0.701,99.259,235933,4,21,r&b pop,87,17
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,Rock Your Body,Justified,Throwbacks,0,NaN,0.892,0.714,4,...,0.000234,0.0521,0.817,100.972,267267,4,79,dance pop,79,77
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,Hot Shot,Throwbacks,0,NaN,0.853,0.606,0,...,0.000000,0.3130,0.654,94.759,227600,4,4,dance reggae pop fusion rap,73,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,The Cadillac Three,spotify:track:4YZFC2igC3mzuktFHSF9mY,Drunk Like You,Bury Me In My Boots,old country,9,NaN,0.543,0.793,0,...,0.000114,0.3630,0.553,140.060,208453,4,38,modern southern rock,55,47
518,Chris Lane,spotify:track:17Vry8h1fUTYg8NL6pHhem,Fix,Girl Problems,old country,9,NaN,0.532,0.883,0,...,0.000000,0.3040,0.700,99.995,190240,4,59,country road contemporary rock modern,59,59
519,Travis Tritt,spotify:track:3oW6SWwGqiZSPTiAp7ZQoH,It's A Great Day To Be Alive,Down The Road I Go,old country,9,NaN,0.595,0.653,7,...,0.000000,0.2640,0.542,128.930,241533,4,75,country road contemporary outlaw,64,69
520,Alan Jackson,spotify:track:2LbOTztU3giUjjYmgtKgj3,Chattahoochee - Extended Mix,16 Biggest Hits,old country,9,NaN,0.579,0.801,0,...,0.000076,0.0821,0.972,174.755,237267,4,56,country,72,61


In [34]:
label_columns = ['track_track_name', 'artist_genre']
check_df[label_columns]

,track_track_name,artist_genre
0,Lose Control (feat. Ciara & Fat Man Scoop),dance urban hop hip virginia soul r&b pop cont...
1,Toxic,dance pop
2,Crazy In Love,r&b pop
3,Rock Your Body,dance pop
4,It Wasn't Me,dance reggae pop fusion rap
...,...,...
517,Drunk Like You,modern southern rock
518,Fix,country road contemporary rock modern
519,It's A Great Day To Be Alive,country road contemporary outlaw
520,Chattahoochee - Extended Mix,country


In [35]:
# Combine 5 fields into one
# string_field = check_df.track_track_name.str.cat(" " + check_df.artist_genre)
string_field = check_df.artist_genre
string_field = string_field.replace({"r\&b": "rhythm blues"}, regex = True)
string_field = string_field.replace({"[^A-Za-z ]+": ""}, regex = True)
#print('last song: ',string_field.tail())
# test_song = string_field[0:int(len(string_field)*0.4)]
# print(test_song)
#fraction = 0.0
#string_field = string_field[int(len(string_field)*fraction):]
string_field

0      dance urban hop hip virginia soul rhythm blues...
1                                              dance pop
2                                       rhythm blues pop
3                                              dance pop
4                            dance reggae pop fusion rap
                             ...                        
517                                 modern southern rock
518                country road contemporary rock modern
519                     country road contemporary outlaw
520                                              country
521                               country outlaw redneck
Name: artist_genre, Length: 522, dtype: object

In [36]:
check_df

,track_artist_name,track_track_uri,track_track_name,track_album_name,playlist_name,playlist_pid,playlist_description,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),The Cookbook,Throwbacks,0,NaN,0.904,0.813,4,...,0.006970,0.0471,0.810,125.461,226864,4,69,dance urban hop hip virginia soul r&b pop cont...,72,62
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Toxic,In The Zone,Throwbacks,0,NaN,0.774,0.838,5,...,0.025000,0.2420,0.924,143.040,198800,4,84,dance pop,80,78
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,0,NaN,0.664,0.758,2,...,0.000000,0.0598,0.701,99.259,235933,4,21,r&b pop,87,17
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,Rock Your Body,Justified,Throwbacks,0,NaN,0.892,0.714,4,...,0.000234,0.0521,0.817,100.972,267267,4,79,dance pop,79,77
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,Hot Shot,Throwbacks,0,NaN,0.853,0.606,0,...,0.000000,0.3130,0.654,94.759,227600,4,4,dance reggae pop fusion rap,73,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,The Cadillac Three,spotify:track:4YZFC2igC3mzuktFHSF9mY,Drunk Like You,Bury Me In My Boots,old country,9,NaN,0.543,0.793,0,...,0.000114,0.3630,0.553,140.060,208453,4,38,modern southern rock,55,47
518,Chris Lane,spotify:track:17Vry8h1fUTYg8NL6pHhem,Fix,Girl Problems,old country,9,NaN,0.532,0.883,0,...,0.000000,0.3040,0.700,99.995,190240,4,59,country road contemporary rock modern,59,59
519,Travis Tritt,spotify:track:3oW6SWwGqiZSPTiAp7ZQoH,It's A Great Day To Be Alive,Down The Road I Go,old country,9,NaN,0.595,0.653,7,...,0.000000,0.2640,0.542,128.930,241533,4,75,country road contemporary outlaw,64,69
520,Alan Jackson,spotify:track:2LbOTztU3giUjjYmgtKgj3,Chattahoochee - Extended Mix,16 Biggest Hits,old country,9,NaN,0.579,0.801,0,...,0.000076,0.0821,0.972,174.755,237267,4,56,country,72,61


In [37]:
for i in string_field:
    print(set(i.split()))

{'dance', 'urban', 'hop', 'hip', 'soul', 'blues', 'pop', 'rhythm', 'contemporary', 'neo', 'rap', 'virginia'}
{'dance', 'pop'}
{'blues', 'pop', 'rhythm'}
{'dance', 'pop'}
{'dance', 'reggae', 'pop', 'fusion', 'rap'}
{'carolina', 'dance', 'south', 'urban', 'hop', 'hip', 'blues', 'pop', 'rhythm', 'contemporary', 'atl', 'rap'}
{'carolina', 'dance', 'south', 'urban', 'hop', 'hip', 'blues', 'pop', 'rhythm', 'contemporary', 'atl', 'rap'}
{'carolina', 'dance', 'south', 'urban', 'hop', 'hip', 'blues', 'pop', 'rhythm', 'contemporary', 'atl', 'rap'}
{'dance', 'pop', 'group', 'girl'}
{'dance', 'urban', 'blues', 'pop', 'rhythm', 'contemporary', 'group', 'girl'}
{'old', 'atlanta', 'south', 'hop', 'hip', 'dirty', 'school', 'southern', 'atl', 'rap'}
{'latin', 'canadian', 'dance', 'pop'}
{'dance', 'pop', 'postteen'}
{'dance', 'pop', 'postteen'}
{'dance', 'pop', 'postteen'}
{'dance', 'urban', 'hip', 'blues', 'pop', 'rhythm', 'contemporary', 'rap'}
{'dance', 'urban', 'hop', 'hip', 'southern', 'blues', 'po

In [38]:
#check_df['All_text'] = check_df.track_track_name.str.cat(" " + check_df.artist_genre)
check_df['All_text'] = check_df.artist_genre
check_df

,track_artist_name,track_track_uri,track_track_name,track_album_name,playlist_name,playlist_pid,playlist_description,danceability,energy,key,...,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity,All_text
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),The Cookbook,Throwbacks,0,NaN,0.904,0.813,4,...,0.0471,0.810,125.461,226864,4,69,dance urban hop hip virginia soul r&b pop cont...,72,62,dance urban hop hip virginia soul r&b pop cont...
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Toxic,In The Zone,Throwbacks,0,NaN,0.774,0.838,5,...,0.2420,0.924,143.040,198800,4,84,dance pop,80,78,dance pop
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,0,NaN,0.664,0.758,2,...,0.0598,0.701,99.259,235933,4,21,r&b pop,87,17,r&b pop
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,Rock Your Body,Justified,Throwbacks,0,NaN,0.892,0.714,4,...,0.0521,0.817,100.972,267267,4,79,dance pop,79,77,dance pop
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,Hot Shot,Throwbacks,0,NaN,0.853,0.606,0,...,0.3130,0.654,94.759,227600,4,4,dance reggae pop fusion rap,73,1,dance reggae pop fusion rap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,The Cadillac Three,spotify:track:4YZFC2igC3mzuktFHSF9mY,Drunk Like You,Bury Me In My Boots,old country,9,NaN,0.543,0.793,0,...,0.3630,0.553,140.060,208453,4,38,modern southern rock,55,47,modern southern rock
518,Chris Lane,spotify:track:17Vry8h1fUTYg8NL6pHhem,Fix,Girl Problems,old country,9,NaN,0.532,0.883,0,...,0.3040,0.700,99.995,190240,4,59,country road contemporary rock modern,59,59,country road contemporary rock modern
519,Travis Tritt,spotify:track:3oW6SWwGqiZSPTiAp7ZQoH,It's A Great Day To Be Alive,Down The Road I Go,old country,9,NaN,0.595,0.653,7,...,0.2640,0.542,128.930,241533,4,75,country road contemporary outlaw,64,69,country road contemporary outlaw
520,Alan Jackson,spotify:track:2LbOTztU3giUjjYmgtKgj3,Chattahoochee - Extended Mix,16 Biggest Hits,old country,9,NaN,0.579,0.801,0,...,0.0821,0.972,174.755,237267,4,56,country,72,61,country


In [39]:
# Imports
from sklearn.cluster import KMeans
from sklearn import metrics 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [40]:
# import nltk
# from nltk.stem import WordNetLemmatizer

In [41]:
# stopwords = nltk.corpus.stopwords.words('english')
# lemmatizer = WordNetLemmatizer()
# nltk.download('stopwords')

In [42]:
check_df.keys()

Index(['track_artist_name', 'track_track_uri', 'track_track_name',
       'track_album_name', 'playlist_name', 'playlist_pid',
       'playlist_description', 'danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature', 'song_popularity',
       'artist_genre', 'artist_popularity', 'album_popularity', 'All_text'],
      dtype='object')

In [43]:
song_features = ['danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'song_popularity', 'artist_popularity', 'album_popularity']
df_song_features = check_df[song_features]

In [44]:
df_song_features

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,song_popularity,artist_popularity,album_popularity
0,0.904,0.813,4,-7.105,0.1210,0.03110,0.006970,0.0471,0.810,125.461,69,72,62
1,0.774,0.838,5,-3.914,0.1140,0.02490,0.025000,0.2420,0.924,143.040,84,80,78
2,0.664,0.758,2,-6.583,0.2100,0.00238,0.000000,0.0598,0.701,99.259,21,87,17
3,0.892,0.714,4,-6.055,0.1410,0.20100,0.000234,0.0521,0.817,100.972,79,79,77
4,0.853,0.606,0,-4.596,0.0713,0.05610,0.000000,0.3130,0.654,94.759,4,73,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,0.543,0.793,0,-5.905,0.0403,0.22500,0.000114,0.3630,0.553,140.060,38,55,47
518,0.532,0.883,0,-2.743,0.0352,0.09510,0.000000,0.3040,0.700,99.995,59,59,59
519,0.595,0.653,7,-4.935,0.0265,0.24600,0.000000,0.2640,0.542,128.930,75,64,69
520,0.579,0.801,0,-7.454,0.0449,0.12100,0.000076,0.0821,0.972,174.755,56,72,61


In [45]:
song_features_array = np.array(df_song_features)

In [46]:
from sklearn.decomposition import TruncatedSVD

In [47]:
vectorizer = TfidfVectorizer(stop_words='english')

X_train = vectorizer.fit_transform(string_field)
print(X_train[0])
x_train_array = X_train.toarray()

svd = TruncatedSVD(n_components=1)
#print(type(svd))
text_X_train = svd.fit_transform(X_train)
#print(text_X_train)
#print(text_X_train.shape)

for index, value in enumerate(text_X_train):
    print(string_field[index], value)

print(text_X_train)
#print(x_train_array)
print(x_train_array.shape)
print(x_train_array[0][44])
f_xtrain_nosvp = np.concatenate((song_features_array, x_train_array), axis=1)
print('xtrain_nosvp', f_xtrain_nosvp.shape)
f_xtrain = np.concatenate((song_features_array, text_X_train), axis=1)
print(f_xtrain)
print(f_xtrain.shape)

#print(X_train)
#print(f"n_samples: {X_train.shape[0]}, n_features: {X_train.shape[1]}")

  (0, 160)	0.33064799790423094
  (0, 204)	0.2344858895583595
  (0, 52)	0.2795697500236954
  (0, 187)	0.13422710385996092
  (0, 26)	0.23574558407778073
  (0, 210)	0.2423888486082042
  (0, 236)	0.33064799790423094
  (0, 266)	0.48388274154583755
  (0, 110)	0.26690604703561915
  (0, 112)	0.2708902786789246
  (0, 262)	0.3071810358033194
  (0, 55)	0.2067186453397504
dance urban hop hip virginia soul rhythm blues pop contemporary rap neo [0.49038904]
dance pop [0.72409486]
rhythm blues pop [0.495341]
dance pop [0.72409486]
dance reggae pop fusion rap [0.40833619]
carolina dance south urban hop hip rhythm blues pop contemporary rap atl [0.50498025]
carolina dance south urban hop hip rhythm blues pop contemporary rap atl [0.50498025]
carolina dance south urban hop hip rhythm blues pop contemporary rap atl [0.50498025]
dance pop group girl [0.44806366]
dance urban rhythm blues pop contemporary group girl [0.51926163]
old atlanta south hop hip dirty school southern rap atl [0.13890203]
latin cana

In [48]:
print(f"{X_train.nnz / np.prod(X_train.shape):.3f}")
print(f"About {X_train.nnz / np.prod(X_train.shape)*100:.3f}\% of the entries in the matrix are non zero")

0.016
About 1.637\% of the entries in the matrix are non zero


In [49]:
true_k = 10

model = KMeans(n_clusters=true_k, init='k-means++', max_iter=500, n_init=100)
model.fit(f_xtrain_nosvp)
#print(model.labels_)
#print(len(model.labels_))

for index, value in enumerate(model.labels_):
    print(string_field[index], value)
    if index == 10:
        break
print('Top terms per cluster')
order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names_out()
#print(terms)
for i in range(true_k):
    print(f"Cluster {i}: ", end="")
    for ind in order_centroids[i,:10]:
        print(f"{terms[ind]} ", end="")
    print()

dance urban hop hip virginia soul rhythm blues pop contemporary rap neo 5
dance pop 5
rhythm blues pop 0
dance pop 9
dance reggae pop fusion rap 0
carolina dance south urban hop hip rhythm blues pop contemporary rap atl 9
carolina dance south urban hop hip rhythm blues pop contemporary rap atl 9
carolina dance south urban hop hip rhythm blues pop contemporary rap atl 9
dance pop group girl 4
dance urban rhythm blues pop contemporary group girl 5
old atlanta south hop hip dirty school southern rap atl 9
Top terms per cluster
Cluster 0: america anthemfolk americana area aesthetic adult acoustic ambient album alternative 
Cluster 1: america anthemfolk area americana aesthetic acoustic adult ambient album melancholia 
Cluster 2: america anthemfolk americana area aesthetic adult acoustic ambient album alternative 
Cluster 3: america anthemfolk area americana aesthetic adult acoustic ambient alternative alt 
Cluster 4: america anthemfolk area americana aesthetic adult ambient acoustic altern

In [50]:
true_k = 10

model = KMeans(n_clusters=true_k, init='k-means++', max_iter=500, n_init=100)
model.fit(X_train)
#print(model.labels_)
#print(len(model.labels_))

for index, value in enumerate(model.labels_):
    print(string_field[index], value)
    if index == 10:
        break
print('Top terms per cluster')
order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names_out()
#print(terms)
for i in range(true_k):
    print(f"Cluster {i}: ", end="")
    for ind in order_centroids[i,:10]:
        print(f"{terms[ind]} ", end="")
    print()

dance urban hop hip virginia soul rhythm blues pop contemporary rap neo 2
dance pop 5
rhythm blues pop 7
dance pop 5
dance reggae pop fusion rap 5
carolina dance south urban hop hip rhythm blues pop contemporary rap atl 2
carolina dance south urban hop hip rhythm blues pop contemporary rap atl 2
carolina dance south urban hop hip rhythm blues pop contemporary rap atl 2
dance pop group girl 5
dance urban rhythm blues pop contemporary group girl 7
old atlanta south hop hip dirty school southern rap atl 2
Top terms per cluster
Cluster 0: wave permanent rock alternative metal funk new grunge synthpop romantic 
Cluster 1: modern rock stomp pop alternative indie holler chamber baroque welsh 
Cluster 2: hip hop rap miami dance pop atl urban contemporary rhythm 
Cluster 3: kpop group boy girl pop krap german gaze garage filter 
Cluster 4: classic album hard rock metal invasion british beatlesque progressive glam 
Cluster 5: dance pop rap postteen art funky neo uk australian mellow 
Cluster 6: 

In [51]:
true_k = 10

model = KMeans(n_clusters=true_k, init='k-means++', max_iter=500, n_init=100)
model.fit(f_xtrain)
#print(model.labels_)
#print(len(model.labels_))

for index, value in enumerate(model.labels_):
    print(string_field[index], text_X_train[index], value)
    if index == 10:
        break
    
print('Top terms per cluster')
order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names_out()
#print(terms)
for i in range(true_k):
    print(f"Cluster {i}: ", end="")
    for ind in order_centroids[i,:10]:
        print(f"{terms[ind]} ", end="")
    print()

dance urban hop hip virginia soul rhythm blues pop contemporary rap neo [0.49038904] 3
dance pop [0.72409486] 3
rhythm blues pop [0.495341] 6
dance pop [0.72409486] 5
dance reggae pop fusion rap [0.40833619] 6
carolina dance south urban hop hip rhythm blues pop contemporary rap atl [0.50498025] 5
carolina dance south urban hop hip rhythm blues pop contemporary rap atl [0.50498025] 5
carolina dance south urban hop hip rhythm blues pop contemporary rap atl [0.50498025] 5
dance pop group girl [0.44806366] 1
dance urban rhythm blues pop contemporary group girl [0.51926163] 3
old atlanta south hop hip dirty school southern rap atl [0.13890203] 5
Top terms per cluster
Cluster 0: america anthemfolk americana area aesthetic adult acoustic ambient album alternative 
Cluster 1: america anthemfolk area americana aesthetic adult ambient acoustic art alternative 
Cluster 2: america anthemfolk area americana aesthetic adult acoustic ambient alternative alt 
Cluster 3: america anthemfolk area america

In [ ]:
song_features_array

# Fuzzy Clustering

## Fuzzy clustering algorithm (with no text features)

In [ ]:
#check_df
#df_song_features

In [61]:
df = df_song_features
#df

In [62]:
# Specify INPUT songs list

input_songs_indices = [1, 212, 213, 214, 215, 216, 217, 218, 200, 300, 400, 500, 514]

input_songs_data = df.loc[input_songs_indices]
print('PLAYLIST OF INPUT SONGS ARE:')
print()
print(check_df['playlist_name'][input_songs_indices])
#print(check_df['track_track_name'][input_songs_indices])

PLAYLIST OF INPUT SONGS ARE:

1                Throwbacks
212                     mat
213                     mat
214                     mat
215                     mat
216                     mat
217                     mat
218                     mat
200                     mat
300                     90s
400    I Put A Spell On You
500                     BOP
514            old country 
Name: playlist_name, dtype: object


In [63]:
# Fuzzy Clustering algorithm (no text features)

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import skfuzzy as fuzz

# Step 1: Load the song data
song_data = df

# Step 2: Define the input songs
#input_songs = ['White Christmas', 'Let It Snow! Let It Snow! Let It Snow!', 'Jingle Bell Rock', 'Winter Wonderland']

# Step 4: Preprocess numerical features
numerical_features = ['danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'song_popularity', 'artist_popularity', 'album_popularity'] #['artist_fraction', 'playlist_description_encoded']  # Replace with the actual numerical feature column names
all_songs_numerical = song_data[numerical_features]
input_songs_numerical = input_songs_data[numerical_features]

# Step 5: Preprocess categorical features
#categorical_features = ['track_track_name', 'playlist_name']  # Replace with the actual categorical feature column names
#all_songs_categorical = song_data[categorical_features]
#input_songs_categorical = input_songs_data[categorical_features]

# Step 6: Perform one-hot encoding on categorical features
encoder = OneHotEncoder(sparse_output=False)
#all_songs_categorical_encoded = encoder.fit_transform(all_songs_categorical)
#input_songs_categorical_encoded = encoder.transform(input_songs_categorical)

# Step 7: Combine numerical and categorical features
all_songs_combined = all_songs_numerical #np.concatenate((all_songs_numerical, all_songs_categorical_encoded), axis=1)
input_songs_combined = input_songs_numerical #np.concatenate((input_songs_numerical, input_songs_categorical_encoded), axis=1)

# Step 8: Scale the feature values
scaler = MinMaxScaler()
X_combined = scaler.fit_transform(all_songs_combined)
input_songs_combined_scaled = scaler.transform(input_songs_combined)

# Step 9: Train the fuzzy clustering model
num_clusters = 7
cntr, u, _, _, _, _, _ = fuzz.cluster.cmeans(X_combined.T, num_clusters, m=1.00001, error=0.000005, maxiter=50000)

# Step 10: Predict the degree of membership for all songs
all_songs_membership = fuzz.cluster.cmeans_predict(X_combined.T, cntr, 2, error=0.000005, maxiter=50000)[0]

# Step 11: Predict the degree of membership for the input songs
input_songs_membership = fuzz.cluster.cmeans_predict(input_songs_combined_scaled.T, cntr, 2, error=0.000005, maxiter=50000)[0]


In [64]:
print('membership matrix of input songs into different clusters:')
#input_songs_membership 

membership matrix of input songs into different clusters:


In [65]:
print('membership matrix of all songs into different clusters:')
#all_songs_membership 

membership matrix of all songs into different clusters:


In [66]:
# membership cluster ranking for all songs(clusters get distant from left to right)

all_songs_membership_array = []
for song_membership in all_songs_membership.T:
    all_sorted_indices = np.argsort(song_membership)[::-1]
    all_songs_membership_array.append(all_sorted_indices)

print('membership cluster ranking for all songs(clusters get distant from left to right):')
#all_songs_membership_array

membership cluster ranking for all songs(clusters get distant from left to right):


In [67]:
# cluster ranking for each input song
print('cluster ranking for each input tarck:')
print('')
i = 0
cluster_num_string_array = []
sorted_indices_array = []

for input_membership_array in input_songs_membership.T:
    sorted_indices = np.argsort(input_membership_array)[::-1]
    sorted_indices_array.append(sorted_indices)
    
    cluster_num_string = "Cluster ranks for Input Track {}:    ".format(i)
    cluster_num_string_array.append(cluster_num_string)
    
    i=i+1
    
test_encoding = tuple(zip(cluster_num_string_array , sorted_indices_array))

for cluster_rankings in test_encoding:
    cluster_label = cluster_rankings[0]
    cluster_array = cluster_rankings[1].tolist()
    print(cluster_label, cluster_array)

cluster ranking for each input tarck:

Cluster ranks for Input Track 0:     [0, 5, 4, 6, 1, 2, 3]
Cluster ranks for Input Track 1:     [3, 6, 2, 1, 4, 5, 0]
Cluster ranks for Input Track 2:     [3, 4, 6, 5, 1, 2, 0]
Cluster ranks for Input Track 3:     [6, 3, 5, 0, 1, 2, 4]
Cluster ranks for Input Track 4:     [3, 1, 6, 5, 0, 2, 4]
Cluster ranks for Input Track 5:     [3, 6, 5, 1, 4, 2, 0]
Cluster ranks for Input Track 6:     [1, 6, 0, 3, 2, 4, 5]
Cluster ranks for Input Track 7:     [6, 1, 0, 3, 4, 2, 5]
Cluster ranks for Input Track 8:     [3, 4, 5, 6, 2, 1, 0]
Cluster ranks for Input Track 9:     [5, 4, 0, 6, 2, 3, 1]
Cluster ranks for Input Track 10:     [5, 4, 6, 0, 3, 1, 2]
Cluster ranks for Input Track 11:     [0, 5, 4, 6, 2, 1, 3]
Cluster ranks for Input Track 12:     [4, 5, 2, 3, 0, 6, 1]


In [68]:
# final cluster ranking by considering distance of all songs to all clusters

input_songs_membership_sum = np.sum(input_songs_membership.T, axis=0)
input_songs_membership_sum

input_songs_membership_sorted_indices = np.argsort(input_songs_membership_sum)[::-1]
print('Final cluster ranking by considering distance from each input track to each cluster center:')
print('')
print(input_songs_membership_sorted_indices)

Final cluster ranking by considering distance from each input track to each cluster center:

[3 5 6 4 0 1 2]


In [72]:
# finding best matching track

refinement = 1

target_array = input_songs_membership_sorted_indices[0:refinement]

matching_index = None

# Iterating over the array list
for index, arr in enumerate(all_songs_membership_array):
    # Checking if the first two entries match

    if np.array_equal(arr[:refinement], target_array):
        matching_index = index
        #break  # Exit the loop after finding the first match

# Printing the matching index
    if matching_index is not None:
        #print("Matching index:", matching_index)
        #print("Matching Track:", check_df["track_track_name"][matching_index])
        print("Playlist of matching Track:", check_df['playlist_name'][matching_index])
        
        print('')
    #else:
        #print("No match found.")
    matching_index = None
if matching_index is None:
    print("No match found.")

Playlist of matching Track: Throwbacks

Playlist of matching Track: korean 

Playlist of matching Track: korean 

Playlist of matching Track: korean 

Playlist of matching Track: korean 

Playlist of matching Track: korean 

Playlist of matching Track: korean 

Playlist of matching Track: mat

Playlist of matching Track: mat

Playlist of matching Track: mat

Playlist of matching Track: mat

Playlist of matching Track: mat

Playlist of matching Track: mat

Playlist of matching Track: mat

Playlist of matching Track: mat

Playlist of matching Track: mat

Playlist of matching Track: mat

Playlist of matching Track: mat

Playlist of matching Track: mat

Playlist of matching Track: mat

Playlist of matching Track: mat

Playlist of matching Track: mat

Playlist of matching Track: mat

Playlist of matching Track: mat

Playlist of matching Track: mat

Playlist of matching Track: mat

Playlist of matching Track: mat

Playlist of matching Track: mat

Playlist of matching Track: mat

Playlist of 

## Fuzzy clustering algorithm (with text features)

In [52]:
# DF Preparation (with text_data)

numerical_features = ['danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'song_popularity', 'artist_popularity', 'album_popularity']

all_features = numerical_features + ['text_data']
df_with_text = pd.DataFrame(np.concatenate((df_song_features, text_X_train), axis=1), columns=all_features)

df = df_with_text

In [53]:
# Specify INPUT songs list

input_songs_indices = [1, 212, 213, 214, 215, 216, 217, 218, 200, 300, 400, 500, 514]

input_songs_data = df.loc[input_songs_indices]
print('PLAYLIST OF INPUT SONGS ARE:')
print()
print(check_df['playlist_name'][input_songs_indices])
#print(check_df['track_track_name'][input_songs_indices])

PLAYLIST OF INPUT SONGS ARE:

1                Throwbacks
212                     mat
213                     mat
214                     mat
215                     mat
216                     mat
217                     mat
218                     mat
200                     mat
300                     90s
400    I Put A Spell On You
500                     BOP
514            old country 
Name: playlist_name, dtype: object


In [54]:
# Fuzzy Clustering algorithm (with text features)

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import skfuzzy as fuzz

# Step 1: Load the song data
song_data = df

# Step 2: Define the input songs
#input_songs = ['White Christmas', 'Let It Snow! Let It Snow! Let It Snow!', 'Jingle Bell Rock', 'Winter Wonderland']

# Step 4: Preprocess numerical features
numerical_features = ['danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'song_popularity', 'artist_popularity', 'album_popularity', 'text_data'] #['artist_fraction', 'playlist_description_encoded']  # Replace with the actual numerical feature column names

all_songs_numerical = song_data[numerical_features]
input_songs_numerical = input_songs_data[numerical_features]

# Step 5: Preprocess categorical features
#categorical_features = ['text_data']  # Replace with the actual categorical feature column names
#all_songs_categorical = song_data[categorical_features]
#input_songs_categorical = input_songs_data[categorical_features]

# Step 6: Perform one-hot encoding on categorical features
encoder = OneHotEncoder(sparse_output=False)
#all_songs_categorical_encoded = encoder.fit_transform(all_songs_categorical)
#input_songs_categorical_encoded = encoder.transform(input_songs_categorical)

# Step 7: Combine numerical and categorical features
all_songs_combined = all_songs_numerical #np.concatenate((all_songs_numerical, all_songs_categorical_encoded), axis=1)
input_songs_combined = input_songs_numerical #np.concatenate((input_songs_numerical, input_songs_categorical_encoded), axis=1)

# Step 8: Scale the feature values
scaler = MinMaxScaler()
X_combined = scaler.fit_transform(all_songs_combined)
input_songs_combined_scaled = scaler.transform(input_songs_combined)

# Step 9: Train the fuzzy clustering model
num_clusters = 7
cntr, u, _, _, _, _, _ = fuzz.cluster.cmeans(X_combined.T, num_clusters, m=1.00001, error=0.000005, maxiter=50000)

# Step 10: Predict the degree of membership for all songs
all_songs_membership = fuzz.cluster.cmeans_predict(X_combined.T, cntr, 2, error=0.000005, maxiter=50000)[0]

# Step 11: Predict the degree of membership for the input songs
input_songs_membership = fuzz.cluster.cmeans_predict(input_songs_combined_scaled.T, cntr, 2, error=0.000005, maxiter=50000)[0]


In [55]:
print('membership matrix of input songs into different clusters:')
#input_songs_membership 

membership matrix of input songs into different clusters:


In [56]:
print('membership matrix of all songs into different clusters:')
#all_songs_membership

membership matrix of all songs into different clusters:


In [57]:
# membership cluster ranking for all songs(clusters get distant from left to right)

all_songs_membership_array = []
for song_membership in all_songs_membership.T:
    all_sorted_indices = np.argsort(song_membership)[::-1]
    all_songs_membership_array.append(all_sorted_indices)

print('membership cluster ranking for all songs(clusters get distant from left to right):')
#all_songs_membership_array

membership cluster ranking for all songs(clusters get distant from left to right):


In [58]:
# cluster ranking for each input song
print('cluster ranking for each input tarck:')
print('')
i = 0
cluster_num_string_array = []
sorted_indices_array = []

for input_membership_array in input_songs_membership.T:
    sorted_indices = np.argsort(input_membership_array)[::-1]
    sorted_indices_array.append(sorted_indices)
    
    cluster_num_string = "Cluster ranks for Input Track {}:    ".format(i)
    cluster_num_string_array.append(cluster_num_string)
    
    i=i+1
    
test_encoding = tuple(zip(cluster_num_string_array , sorted_indices_array))

for cluster_rankings in test_encoding:
    cluster_label = cluster_rankings[0]
    cluster_array = cluster_rankings[1].tolist()
    print(cluster_label, cluster_array)

cluster ranking for each input tarck:

Cluster ranks for Input Track 0:     [5, 0, 2, 1, 4, 3, 6]
Cluster ranks for Input Track 1:     [3, 1, 2, 4, 6, 0, 5]
Cluster ranks for Input Track 2:     [3, 0, 6, 4, 2, 1, 5]
Cluster ranks for Input Track 3:     [2, 3, 1, 4, 5, 0, 6]
Cluster ranks for Input Track 4:     [3, 4, 2, 1, 0, 5, 6]
Cluster ranks for Input Track 5:     [3, 2, 4, 0, 1, 6, 5]
Cluster ranks for Input Track 6:     [4, 2, 1, 3, 5, 6, 0]
Cluster ranks for Input Track 7:     [2, 4, 5, 3, 1, 0, 6]
Cluster ranks for Input Track 8:     [3, 0, 2, 6, 1, 4, 5]
Cluster ranks for Input Track 9:     [0, 5, 2, 1, 6, 3, 4]
Cluster ranks for Input Track 10:     [0, 2, 5, 3, 1, 6, 4]
Cluster ranks for Input Track 11:     [0, 5, 2, 1, 6, 4, 3]
Cluster ranks for Input Track 12:     [6, 0, 1, 3, 2, 5, 4]


In [59]:
# final cluster ranking by considering distance of all songs to all clusters

input_songs_membership_sum = np.sum(input_songs_membership.T, axis=0)
input_songs_membership_sum

input_songs_membership_sorted_indices = np.argsort(input_songs_membership_sum)[::-1]
print('Final cluster ranking by considering distance from each input track to each cluster center:')
print('')
print(input_songs_membership_sorted_indices)

Final cluster ranking by considering distance from each input track to each cluster center:

[3 0 2 4 5 1 6]


In [74]:
# finding the best matching track

refinement = 2

target_array = input_songs_membership_sorted_indices[0:refinement]

matching_index = None

# Iterating over the array list
for index, arr in enumerate(all_songs_membership_array):
    # Checking if the first two entries match

    if np.array_equal(arr[:refinement], target_array):
        matching_index = index
        #break  # Exit the loop after finding the first match

# Printing the matching index
    if matching_index is not None:
        #print("Matching index:", matching_index)
        #print("Matching Track:", check_df["track_track_name"][matching_index])
        print("Playlist of matching Track:", check_df['playlist_name'][matching_index])
        
        print('')
    #else:
        #print("No match found.")
    matching_index = None
if matching_index is None:
    print("No match found.")

Playlist of matching Track: Throwbacks

Playlist of matching Track: korean 

Playlist of matching Track: mat

Playlist of matching Track: mat

Playlist of matching Track: mat

Playlist of matching Track: mat

Playlist of matching Track: Wedding

No match found.
